In [58]:
#Script to run the perturbation climate EBM based on Gregory regressions of TRACMIP data. 
#Start with all rapid adjustments/feedbacks combined. 
#Regressions done in "GregoryTRACMIP.ipynb".

#Version with G = 0 (shouldn't be any ocean heat uptake)
#Just using a single number 0 for G input should be fine--only added twice, dimension shouldn't matter
#Started 19 November 2019

#10 December 2019: add an instantaneous radiative forcing run, 
#calculated as the difference between the effective forcing and 
#the sum of the rapid adjustments

#Note: in some cases runs completed in the background but output did not print because 
#window had been closed. 

In [2]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd #necessary for new dimension in concat

/usr/local/python/anaconda3/envs/pangeo3.7/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/usr/local/python/anaconda3/envs/pangeo3.7/lib/python3.6/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [3]:
#Perturbation version of the EBM with forcing, feedback, ocean heat uptake, and control temperature inputs
# 5-21-19: modified to only diffuse anomalous MSE rather than absolute. 

# Inputs:
# Rf: radiative forcing as a function of latitude
# lamda: feedback as a function of latitude (misspelled because "lambda" has special meaning in Python)
# G: ocean heat uptake (i.e. change in net surface heat fluxes) as a function of latitude
# T_control: temperature from the control climate (do things based on perturbations ot this)
# lats: latitudes for the model grid
# latBounds: can be specified, or automatically calculated based on given lats (put in default based on TRACMIP climatology common grid)
# anom: whether to diffuse anomalous rather than absolute MSE (papers and code from Aaron suggest it should be this way)
#
# Inputs with defalt values:
# D: Diffusion coefficient (m^2/s)
# dt: time step (arbitrary units)
# ps: surface pressure (Pa)
# RH: relative humidity (fraction)
# C: ocean heat capacity (arbitrary units)
# nmax: maximum number of iterations before giving up if equilibrium not reached
# tol: Max. temperature difference between 2 time steps to define equilibrium (K)
# tempPrint: if true, print the temperatures at selected latitudes when indicating progress
#
# Output: 2 XArray datasets, one on latitude and one on boundary latitude grid
def moistEBM_perturb(Rf, lamda, G, T_control, lats, latBounds='auto',
                  D=1.06e6, dt=1.0e-4, ps=9.8e4, RH=0.8, C=1, nmax=5e5, tol=0.001, tempPrint = False):
    
    #Physical constants
    Cp = 1004.   #Heat capacity of air at constant pressure (J/(kg*K))
    Lv = 2.5e6  #Latent heat vaporization at 0 deg C (J/kg)
    g = 9.8 #Gravitational acceleration (m/s^2)
    r_earth = 6.37e6 #Earth radius (m)
    Rd = 287. #Gas constant for dry air
    Rv = 461. #Gas constant for water vapor
    Lv = 2.5e6 #Latent heat of vaporization
    
    TS = T_control+0 #Initial temperature from control climate (+0 to avoid possibly changing T_control later)
    
    #Set up matrices for numerical integration of diffusion PDE
    #A and B represent transport of heat into grid box from the north and the south, respectively;
    #together they represent a 2nd-order finite-difference second derivative, weighted by latitude. 
    A = np.zeros((len(lats),len(lats)))
    B = np.zeros((len(lats),len(lats)))
    for i in range(1,len(lats)-1):
        A[i,i]=-1
        A[i,i+1]=1
        B[i,i]=1
        B[i,i-1]=-1
        
        
    #Calculate latitude boundaries if not specified
    if latBounds == 'auto':
        latBounds = np.append(-90, lats+(lats[1]-lats[0])/2.)
        
    #Define grid for EBM
    #print(latBounds)
    latDiff = np.diff(latBounds) #Grid spacing 
    latsNB = latBounds[1:len(latBounds)] #Latitudes at North Boundaries of grid boxes
    latsSB = latBounds[0:len(latBounds)-1] #Latitudes at South Boundaries of grid boxes
    lats_rad = lats*np.pi/180. #Convert from degrees to radians
    latsNB_rad = latsNB*np.pi/180.
    latsSB_rad = latsSB*np.pi/180.
    latDiff_rad = latDiff*np.pi/180.
    
    if tempPrint:
        #Print the temperatures at a few selected latitudes while running
        #Floor division to get integer indices
        printIndices=[len(lats)//10, 
                      len(lats)//5, 
                      len(lats)//3, 
                      len(lats)//2, 
                      (2*len(lats))//3, 
                      (4*len(lats))//5, 
                      (9*len(lats))//10]
        print('Latitudes for test temperature printing:')
        for k in range(len(printIndices)):
            print(lats[printIndices[k]])
    
    #Iterate through temperature tendency
    n = 0 #Iterative loop counter
    conv = False #Whether we've reached equilibrium
    while((n < nmax) & (conv == False)):
        #Set up profile of moist static energy
        #Saturation vapor pressure as a function of latitude:        
        es = 611.*np.exp((Lv/Rv)*((1./273.15)-(1./TS))) #Hartmann, 2016, eq. 1.11
        q = (RH*es)/ps*Rd/Rv #Specific humidity (kg/kg): RH = e/es therefore e = RH*es, and q = e/ps*Rd/Rv
        MSE = Cp*TS+Lv*q #Moist static energy at surface
        
        #Calculate climatological MSE and then perturbation by subtracting it out
        es_control = 611.*np.exp((Lv/Rv)*((1./273.15)-(1./T_control)))
        q_control = (RH*es_control)/ps*Rd/Rv
        MSE_control = Cp*T_control+Lv*q_control
        MSE_anom = MSE - MSE_control
         
        #Calculate diffusion/transport of anomalous MSE
        Transport = (D*(ps/g)/(r_earth*r_earth*np.cos(lats_rad)*latDiff_rad*latDiff_rad))*(
                    np.cos(latsNB_rad)*np.dot(A,MSE_anom)-np.cos(latsSB_rad)*np.dot(B,MSE_anom))
        #Diffusion at south boundary
        Transport[0] = (D*(ps/g)/(r_earth*r_earth*np.cos(lats_rad[0])*latDiff_rad[0]*latDiff_rad[0]))*(
                       np.cos(lats_rad[1])*(MSE_anom[2]-MSE_anom[1])-np.cos(lats_rad[0])*(MSE_anom[1]-MSE_anom[0]))
        #Diffusion at north boundary
        Transport[len(lats)-1] = (D*(ps/g)/(r_earth*r_earth*np.cos(lats_rad[len(lats)-1])*latDiff_rad[len(lats)-1]*latDiff_rad[len(lats)-1]))*(
                       np.cos(lats_rad[len(lats)-1])*(MSE_anom[len(lats)-1]-MSE_anom[len(lats)-2])-
                       np.cos(lats_rad[len(lats)-2])*(MSE_anom[len(lats)-2]-MSE_anom[len(lats)-3]))
        
        #Calculate perturbation temperature for feedback calculation (will be 0 on first time step)
        TS_perturb = TS - T_control
        
        #Temperature tendency/time stepping. When equilibrium is reached, the new and old TS cancel
        #and this equation represents energy balance, e.g. equation (1) of Bonan et al. (2018). 
        #Transport out
        TS = TS + dt*(Rf+lamda*TS_perturb+Transport-G)/C
        
        #Check for equilibrium (temperature has stopped changing) and return results if so
        #print(np.max(np.abs(EI-OLR+Transport)))
        if ((np.max(np.abs(Rf+lamda*TS_perturb+Transport-G)) < tol) and (n > 0)):
            conv = True
            print('EBM reached equilibrium in ' + str(n) + ' iterations')
            
            #Calculate the northward transport by integrating the transport/diffusion term northward
            NorthTransport = -np.cumsum(Transport*(2*np.pi*r_earth*r_earth)*(np.sin(latsNB_rad)-np.sin(latsSB_rad)))
            
            #Correct for transport at north pole (caused by numerics)
            N = NorthTransport[-1] #Northward energy transport at NP
            E = N/2.*(1+np.sin(lats*np.pi/180.)) 
            CorrectedTransport = NorthTransport - E
            
            #Versions with 0 appended (for plotting at boundary latitudes)
            NorthTransportBounds = np.append(0, NorthTransport)
            CorrectedTransportBounds = np.append(0, CorrectedTransport)
            
            #Return 2 XArray objects: ds_lat (for variables on the latitude grid) and ds_latBounds (for variables on boundary latitudes)
            ds_lat = xr.Dataset({'CorrectedTransport': (['lat'], CorrectedTransport),
                                 'Diffusion': (['lat'], Transport), 
                                 'MSE': (['lat'], MSE), 
                                 'MSE_anom': (['lat'], MSE_anom),
                                 'NorthTransport': (['lat'], NorthTransport), 
                                 'q': (['lat'], q), 
                                 'TS_perturb': ('lat', TS_perturb),
                                 'TS': (['lat'], TS)}, 
                                coords = {'lat': (['lat'], lats)})
            
#             ds_latBounds = xr.Dataset({'CorrectedTransportBounds': (['lat'], CorrectedTransportBounds), 
#                                        'NorthTransportBounds': (['lat'], NorthTransportBounds)}, 
#                                       coords ={'lat': (['lat'], latBounds)})
            
            return ds_lat
            
        if np.mod(n,6000)==0:
            print('EBM (perturbation) finished iteration ' + str(n))
            if tempPrint:
                print('{:e}, {:e}, {:e}, {:e}, {:e}, {:e}, {:e} K'.format(TS[printIndices[0]], 
                                                                          TS[printIndices[1]],
                                                                          TS[printIndices[2]],
                                                                          TS[printIndices[3]],
                                                                          TS[printIndices[4]],
                                                                          TS[printIndices[5]],
                                                                          TS[printIndices[6]]))
                
        n = n + 1 #Iterate loop counter
        #print(n)
        #Warning message if equilibrium not reached (Will return nothing)
        if(n == nmax):
            #print('WARNING: equilibrium not reached in moist EBM--nothing returned')
            print('WARNING: equilibrium not reached in moist EBM--nans returned')
            
            #Should really return all nans instead
            nanarray = np.ones(len(lats))*np.nan
            ds_lat = xr.Dataset({'CorrectedTransport': (['lat'], nanarray),
                                 'Diffusion': (['lat'], nanarray), 
                                 'MSE': (['lat'], nanarray), 
                                 'MSE_anom': (['lat'], nanarray),
                                 'NorthTransport': (['lat'], nanarray), 
                                 'q': (['lat'], nanarray), 
                                 'TS_perturb': ('lat', nanarray),
                                 'TS': (['lat'], nanarray)}, 
                                coords = {'lat': (['lat'], lats)})
            
            print(ds_lat)
            return ds_lat

In [4]:
#Load climatology data (for climatological temperature) and the Gregory regression output
ds_clim = xr.open_dataset('../nc_revised_20181130/master.nc')
ds_greg = xr.open_dataset('nc_from_xarray/GregoryTotalForcingFeedbackStorage_a4.nc')

In [5]:
models = ['AM2', 'CAM3', 'CAM4', 'CNRM-AM6-DIA-v2', 'ECHAM-6.1', 'ECHAM-6.3',
       'IPSL-CM5A', 'MIROC5', 'MPAS', 'MetUM-GA6-CTL', 'MetUM-GA6-ENT',
       'NorESM2']

In [6]:
#CNRM model had "divide by zero" error for water vapor, so skip it for water vapor kernels
models_noCNRM = ['AM2', 'CAM3', 'CAM4', 'ECHAM-6.1', 'ECHAM-6.3',
       'IPSL-CM5A', 'MIROC5', 'MPAS', 'MetUM-GA6-CTL', 'MetUM-GA6-ENT',
       'NorESM2']

In [7]:
#Skipping NorESM2 as well because it didn't have clear-sky LW output saved
models_10 = ['AM2', 'CAM3', 'CAM4', 'ECHAM-6.1', 'ECHAM-6.3',
       'IPSL-CM5A', 'MIROC5', 'MPAS', 'MetUM-GA6-CTL', 'MetUM-GA6-ENT']

In [8]:
lats = ds_clim.lat.data

In [9]:

#####   RUNS   #####


In [10]:

###   Control Runs (9.6*10^5 diffusivity)   ###


In [11]:
EBM_perturb_results_096e4_dict = dict()
for model in models:
    print('Running perturbation EBM (Aqua4xCO2) for model: ' + model)
    EBM_perturb_results_096e4_dict[model] = moistEBM_perturb(ds_greg['forcing'].sel(model=model).data, 
                                            ds_greg['feedback'].sel(model=model).data,
                                            0, 
                                            ds_clim['ts'].sel(model=model, exp='AquaControl').mean(dim='lon').mean(dim='time').data,
                                            ds_clim.lat.data, D=0.96e6, dt=5.0e-5, nmax=8e5)

Running perturbation EBM (Aqua4xCO2) for model: AM2
EBM (perturbation) finished iteration 0
EBM (perturbation) finished iteration 6000
EBM (perturbation) finished iteration 12000
EBM (perturbation) finished iteration 18000
EBM (perturbation) finished iteration 24000
EBM (perturbation) finished iteration 30000
EBM (perturbation) finished iteration 36000
EBM (perturbation) finished iteration 42000
EBM (perturbation) finished iteration 48000
EBM (perturbation) finished iteration 54000
EBM (perturbation) finished iteration 60000
EBM (perturbation) finished iteration 66000
EBM (perturbation) finished iteration 72000
EBM (perturbation) finished iteration 78000
EBM (perturbation) finished iteration 84000
EBM (perturbation) finished iteration 90000
EBM (perturbation) finished iteration 96000
EBM (perturbation) finished iteration 102000
EBM (perturbation) finished iteration 108000
EBM (perturbation) finished iteration 114000
EBM (perturbation) finished iteration 120000
EBM (perturbation) finish

In [12]:
#Concatenate and save
EBM_perturb_results_096e4 = xr.concat(list(EBM_perturb_results_096e4_dict.values())[:], dim=pd.Index(models, name='model')) 
EBM_perturb_results_096e4.to_netcdf('nc_from_xarray/EBM_perturb_results_noG_096e4.nc')

In [13]:

###   Perturbing Rapid Adjustments And Feedbacks One At A Time   ###


In [14]:
#8-8-19: Having calculated individual rapid adjustments and feedbacks using Gregory regressions, 
#time to perturb them one at a time in the EBM. 
#The perturbations are done as subtractions from the forcing and feedback terms in the control runs above. 

#Naming convention: 
#EBM_results_[adj, fb]_[SW, LW]_[cloud, noncloud, ta, wv, ts]

In [15]:
#Load the results and subset to individual adjustments/feedbacks
#Flip sign for LW feedbacks because kernels are positive upward
ds_SW = xr.open_dataset('nc_from_xarray/GregoryAPRP_a4.nc')
ds_LW_ta = xr.open_dataset('nc_from_xarray/Gregory_kernel_ta_a4.nc')
ds_LW_wv = xr.open_dataset('nc_from_xarray/Gregory_kernel_wv_a4.nc')
ds_LW_ts = xr.open_dataset('nc_from_xarray/Gregory_kernel_ts_a4.nc')
ds_LW_cloud = xr.open_dataset('nc_from_xarray/Gregory_dLWCRE_co_a4.nc')

In [16]:
da_adj_SW_cloud = ds_SW['adj_cloud']
da_adj_SW_noncloud = ds_SW['adj_noncloud']
da_adj_LW_ta = -ds_LW_ta['adj']
da_adj_LW_wv = -ds_LW_wv['adj']
da_adj_LW_cloud = ds_LW_cloud['adj']

In [17]:
da_fb_SW_cloud = ds_SW['fb_cloud']
da_fb_SW_noncloud = ds_SW['fb_noncloud']
da_fb_LW_ta = -ds_LW_ta['fb']
da_fb_LW_wv = -ds_LW_wv['fb']
da_fb_LW_ts = -ds_LW_ts['fb'] #No rapid adjustment for this one
da_fb_LW_cloud = ds_LW_cloud['fb']

In [18]:
#The surface temperature kernel has nans at the edges--duplicate the adjacent values
#to avoid the EBM returning nans
#print(da_fb_LW_ts)
da_fb_LW_ts.loc[dict(lat=-89.5)] = da_fb_LW_ts.loc[dict(lat=-88.5)]
da_fb_LW_ts.loc[dict(lat=89.5)] = da_fb_LW_ts.loc[dict(lat=88.5)]
#print(da_fb_LW_ts)

In [19]:
#Other kernels had weird edge effects at outer 1-2 latitudes; 
#may need to fix this but shouldn't matter much

In [20]:
#Fix edge effects for LW cloud stuff as well

In [21]:
for model in models_10:
    if model in ['CAM3']:
        da_adj_LW_cloud.loc[dict(model=model, lat=-89.5)] = da_adj_LW_cloud.sel(model=model, lat=-87.5)
        da_adj_LW_cloud.loc[dict(model=model, lat=-88.5)] = da_adj_LW_cloud.sel(model=model, lat=-87.5)
        da_adj_LW_cloud.loc[dict(model=model, lat=89.5)] = da_adj_LW_cloud.sel(model=model, lat=87.5)
        da_adj_LW_cloud.loc[dict(model=model, lat=88.5)] = da_adj_LW_cloud.sel(model=model, lat=87.5)
        da_fb_LW_cloud.loc[dict(model=model, lat=-89.5)] = da_fb_LW_cloud.sel(model=model, lat=-87.5)
        da_fb_LW_cloud.loc[dict(model=model, lat=-88.5)] = da_fb_LW_cloud.sel(model=model, lat=-87.5)
        da_fb_LW_cloud.loc[dict(model=model, lat=89.5)] = da_fb_LW_cloud.sel(model=model, lat=87.5)
        da_fb_LW_cloud.loc[dict(model=model, lat=88.5)] = da_fb_LW_cloud.sel(model=model, lat=87.5)
    else:
        da_adj_LW_cloud.loc[dict(model=model, lat=-89.5)] = da_adj_LW_cloud.sel(model=model, lat=-88.5)
        da_adj_LW_cloud.loc[dict(model=model, lat=89.5)] = da_adj_LW_cloud.sel(model=model, lat=88.5)
        da_fb_LW_cloud.loc[dict(model=model, lat=-89.5)] = da_fb_LW_cloud.sel(model=model, lat=-88.5)
        da_fb_LW_cloud.loc[dict(model=model, lat=89.5)] = da_fb_LW_cloud.sel(model=model, lat=88.5)
        

In [65]:
#Calculate the instantaneous rapid adjustment (as residual) and fix edge lats if necessary
f_inst = ds_greg['forcing'] - (ds_SW['adj_cloud'] + ds_SW['adj_noncloud']
                                + ds_LW_cloud['adj'] - ds_LW_wv['adj'] - ds_LW_ta['adj'])
#print(f_inst.isel(lat=0)) #OK, no edge zeros
#print(f_inst) #DataArray--should be fine

<xarray.DataArray (model: 10, lat: 180)>
array([[6.226819, 5.032414, 4.850643, ..., 4.675564, 4.715272, 4.555197],
       [4.828593, 4.828614, 4.749604, ..., 5.027222, 4.951998, 4.952046],
       [3.848936, 4.493804, 4.420323, ..., 4.604786, 4.834987, 4.74915 ],
       ...,
       [4.420551, 4.609495, 4.500985, ..., 4.654716, 4.870671, 4.734898],
       [5.311714, 4.868955, 4.875584, ..., 4.260537, 4.266573, 3.990313],
       [5.338479, 4.732634, 4.759465, ..., 4.618388, 4.602416, 4.809043]])
Coordinates:
  * model    (model) object 'AM2' 'CAM3' ... 'MetUM-GA6-CTL' 'MetUM-GA6-ENT'
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5


In [22]:
#Functions to run the EBM for each model for a given process
#(call with G = 0 here--will fix that problem)

#One for adjustment, one for feedback, possibly one for both
#ds_clim: for climatological temperature
#ds_control: Dataset containing forcing, feedback terms
#da_perturb: DataArray loaded above for the rapid adjustments/feedbacks
#D: diffusivity
#dt, nmax: time step, max number of iterations
#Can enter dt, nmax as dicts, different for each model, instead.
#Try-except since don't want to break backward compatibility
#scale: percentage of the perturbation to apply (default 1 for backward compatibility)
#
#Returns: EBM_results_da: XArray DataArray containing the results for each model. 
#(save to NetCDF separately)
def run_EBM_perturb_adj(ds_clim, ds_control, da_perturb, models, D, dt=9.0e-5, nmax=7e5, scale=1, tempPrint=True):
    EBM_results_dict = dict()
    for model in models: 
        print(model)
        try:
            dt_model = dt[model] #input dict
        except:
            dt_model = dt
        try: 
            nmax_model = nmax[model]
        except: 
            nmax_model = nmax
        EBM_results_dict[model] = moistEBM_perturb(
            ds_control['forcing'].sel(model=model).data - da_perturb.sel(model=model).data * scale, 
            ds_control['feedback'].sel(model=model).data,
            0,
            ds_clim['ts'].sel(model=model, exp='AquaControl').mean(dim='lon').mean(dim='time').data,
            ds_clim.lat.data, 
            D=D, dt=dt_model, nmax=nmax_model, tempPrint=tempPrint
            )
    EBM_results_da = xr.concat(list(EBM_results_dict.values())[:], dim=pd.Index(models, name='model'))
    return EBM_results_da
    
def run_EBM_perturb_fb(ds_clim, ds_control, da_perturb, models, D, dt=9.0e-5, nmax=7e5, scale=1, tempPrint=True):
    EBM_results_dict = dict()
    for model in models:
        print(model)
        try:
            dt_model = dt[model] #input dict
        except:
            dt_model = dt
        try: 
            nmax_model = nmax[model]
        except: 
            nmax_model = nmax
        EBM_results_dict[model] = moistEBM_perturb(
            ds_control['forcing'].sel(model=model).data, 
            ds_control['feedback'].sel(model=model).data - da_perturb.sel(model=model).data * scale,
            0,
            ds_clim['ts'].sel(model=model, exp='AquaControl').mean(dim='lon').mean(dim='time').data,
            ds_clim.lat.data, 
            D=D, dt=dt_model, nmax=nmax_model, tempPrint=tempPrint
            )
    EBM_results_da = xr.concat(list(EBM_results_dict.values())[:], dim=pd.Index(models, name='model'))
    return EBM_results_da

In [23]:
#Run for each case and save to NetCDF files


In [67]:
#New (12-11-19): Inst. CO2 forcing
EBM_results_f_inst = run_EBM_perturb_adj(ds_clim, ds_greg, f_inst, models_10, D=0.96e6)

AM2
Latitudes for test temperature printing:
-71.5
-53.5
-29.5
0.5
30.5
54.5
72.5
EBM (perturbation) finished iteration 0
2.841678e+02, 2.896704e+02, 3.004598e+02, 3.080435e+02, 3.055914e+02, 2.953126e+02, 2.908842e+02 K
EBM (perturbation) finished iteration 6000
2.841430e+02, 2.898527e+02, 3.006004e+02, 3.081500e+02, 3.057605e+02, 2.954413e+02, 2.906329e+02 K
EBM (perturbation) finished iteration 12000
2.842637e+02, 2.899555e+02, 3.006685e+02, 3.081956e+02, 3.058129e+02, 2.955193e+02, 2.907252e+02 K
EBM (perturbation) finished iteration 18000
2.843480e+02, 2.900241e+02, 3.007118e+02, 3.082245e+02, 3.058484e+02, 2.955771e+02, 2.907980e+02 K
EBM (perturbation) finished iteration 24000
2.844040e+02, 2.900698e+02, 3.007408e+02, 3.082441e+02, 3.058730e+02, 2.956175e+02, 2.908493e+02 K
EBM (perturbation) finished iteration 30000
2.844415e+02, 2.901004e+02, 3.007603e+02, 3.082575e+02, 3.058898e+02, 2.956453e+02, 2.908847e+02 K
EBM (perturbation) finished iteration 36000
2.844667e+02, 2.90121

In [69]:
EBM_results_f_inst.to_netcdf('nc_from_xarray/EBM_results_noG_f_inst.nc')

In [70]:
EBM_results_adj_SW_cloud = run_EBM_perturb_adj(ds_clim, ds_greg, da_adj_SW_cloud, models, D=0.96e6)

AM2
Latitudes for test temperature printing:
-71.5
-53.5
-29.5
0.5
30.5
54.5
72.5
EBM (perturbation) finished iteration 0
2.841687e+02, 2.896709e+02, 3.004602e+02, 3.080441e+02, 3.055918e+02, 2.953131e+02, 2.908852e+02 K
EBM (perturbation) finished iteration 6000
2.880476e+02, 2.930907e+02, 3.029571e+02, 3.100528e+02, 3.079088e+02, 2.986788e+02, 2.948464e+02 K
EBM (perturbation) finished iteration 12000
2.910157e+02, 2.955755e+02, 3.046226e+02, 3.112447e+02, 3.093987e+02, 3.010310e+02, 2.977238e+02 K
EBM (perturbation) finished iteration 18000
2.930126e+02, 2.972525e+02, 3.057625e+02, 3.120674e+02, 3.104049e+02, 3.025682e+02, 2.995693e+02 K
EBM (perturbation) finished iteration 24000
2.943398e+02, 2.983751e+02, 3.065372e+02, 3.126307e+02, 3.110825e+02, 3.035776e+02, 3.007641e+02 K
EBM (perturbation) finished iteration 30000
2.952208e+02, 2.991241e+02, 3.070594e+02, 3.130128e+02, 3.115373e+02, 3.042445e+02, 3.015465e+02 K
EBM (perturbation) finished iteration 36000
2.958054e+02, 2.99623

In [72]:
EBM_results_adj_SW_cloud.to_netcdf('nc_from_xarray/EBM_results_noG_adj_sw_cloud.nc')

In [26]:
EBM_results_adj_SW_noncloud = run_EBM_perturb_adj(ds_clim, ds_greg, da_adj_SW_noncloud, models, D=0.96e6)

AM2
Latitudes for test temperature printing:
-71.5
-53.5
-29.5
0.5
30.5
54.5
72.5
EBM (perturbation) finished iteration 0
2.841682e+02, 2.896706e+02, 3.004602e+02, 3.080439e+02, 3.055918e+02, 2.953128e+02, 2.908845e+02 K
EBM (perturbation) finished iteration 6000
2.865959e+02, 2.922255e+02, 3.026975e+02, 3.098125e+02, 3.075127e+02, 2.976845e+02, 2.930960e+02 K
EBM (perturbation) finished iteration 12000
2.889965e+02, 2.942140e+02, 3.040061e+02, 3.107388e+02, 3.086573e+02, 2.995065e+02, 2.953677e+02 K
EBM (perturbation) finished iteration 18000
2.906262e+02, 2.955534e+02, 3.048886e+02, 3.113687e+02, 3.094347e+02, 3.007320e+02, 2.968878e+02 K
EBM (perturbation) finished iteration 24000
2.917106e+02, 2.964504e+02, 3.054880e+02, 3.118005e+02, 3.099623e+02, 3.015489e+02, 2.978895e+02 K
EBM (perturbation) finished iteration 30000
2.924335e+02, 2.970512e+02, 3.058933e+02, 3.120942e+02, 3.103188e+02, 3.020939e+02, 2.985526e+02 K
EBM (perturbation) finished iteration 36000
2.929162e+02, 2.97453

In [27]:
EBM_results_adj_SW_noncloud.to_netcdf('nc_from_xarray/EBM_results_noG_adj_sw_noncloud.nc')

In [28]:
EBM_results_adj_LW_ta = run_EBM_perturb_adj(ds_clim, ds_greg, da_adj_LW_ta, models, D=0.96e6)

AM2
Latitudes for test temperature printing:
-71.5
-53.5
-29.5
0.5
30.5
54.5
72.5
EBM (perturbation) finished iteration 0
2.841682e+02, 2.896705e+02, 3.004601e+02, 3.080439e+02, 3.055918e+02, 2.953128e+02, 2.908846e+02 K
EBM (perturbation) finished iteration 6000
2.862922e+02, 2.918991e+02, 3.024875e+02, 3.097101e+02, 3.074216e+02, 2.976032e+02, 2.931246e+02 K
EBM (perturbation) finished iteration 12000
2.884705e+02, 2.937172e+02, 3.036918e+02, 3.105711e+02, 3.085006e+02, 2.993350e+02, 2.952837e+02 K
EBM (perturbation) finished iteration 18000
2.899781e+02, 2.949597e+02, 3.045094e+02, 3.111554e+02, 3.092271e+02, 3.004860e+02, 2.967095e+02 K
EBM (perturbation) finished iteration 24000
2.909915e+02, 2.957983e+02, 3.050667e+02, 3.115553e+02, 3.097179e+02, 3.012489e+02, 2.976436e+02 K
EBM (perturbation) finished iteration 30000
2.916706e+02, 2.963624e+02, 3.054445e+02, 3.118274e+02, 3.100489e+02, 3.017567e+02, 2.982605e+02 K
EBM (perturbation) finished iteration 36000
2.921254e+02, 2.96741

In [29]:
EBM_results_adj_LW_ta.to_netcdf('nc_from_xarray/EBM_results_noG_adj_lw_ta.nc')

In [30]:
EBM_results_adj_LW_wv = run_EBM_perturb_adj(ds_clim, ds_greg, da_adj_LW_wv, models_noCNRM, D=0.96e6)

AM2
Latitudes for test temperature printing:
-71.5
-53.5
-29.5
0.5
30.5
54.5
72.5
EBM (perturbation) finished iteration 0
2.841682e+02, 2.896707e+02, 3.004603e+02, 3.080439e+02, 3.055918e+02, 2.953128e+02, 2.908845e+02 K
EBM (perturbation) finished iteration 6000
2.870054e+02, 2.926597e+02, 3.030998e+02, 3.100719e+02, 3.076968e+02, 2.978545e+02, 2.932247e+02 K
EBM (perturbation) finished iteration 12000
2.897917e+02, 2.949535e+02, 3.045980e+02, 3.111254e+02, 3.089827e+02, 2.998825e+02, 2.957447e+02 K
EBM (perturbation) finished iteration 18000
2.916485e+02, 2.964764e+02, 3.056017e+02, 3.118436e+02, 3.098637e+02, 3.012615e+02, 2.974510e+02 K
EBM (perturbation) finished iteration 24000
2.928703e+02, 2.974878e+02, 3.062810e+02, 3.123366e+02, 3.104641e+02, 3.021845e+02, 2.985798e+02 K
EBM (perturbation) finished iteration 30000
2.936796e+02, 2.981619e+02, 3.067394e+02, 3.126722e+02, 3.108703e+02, 3.028007e+02, 2.993272e+02 K
EBM (perturbation) finished iteration 36000
2.942179e+02, 2.98612

In [31]:
EBM_results_adj_LW_wv.to_netcdf('nc_from_xarray/EBM_results_noG_adj_lw_wv.nc')

In [32]:
EBM_results_fb_SW_cloud = run_EBM_perturb_fb(ds_clim, ds_greg, da_fb_SW_cloud, models, D=0.96e6)

AM2
Latitudes for test temperature printing:
-71.5
-53.5
-29.5
0.5
30.5
54.5
72.5
EBM (perturbation) finished iteration 0
2.841682e+02, 2.896707e+02, 3.004603e+02, 3.080440e+02, 3.055918e+02, 2.953129e+02, 2.908846e+02 K
EBM (perturbation) finished iteration 6000
2.862139e+02, 2.918202e+02, 3.024664e+02, 3.097425e+02, 3.073690e+02, 2.973406e+02, 2.926808e+02 K
EBM (perturbation) finished iteration 12000
2.874946e+02, 2.928902e+02, 3.032300e+02, 3.103257e+02, 3.080445e+02, 2.983054e+02, 2.938122e+02 K
EBM (perturbation) finished iteration 18000
2.880528e+02, 2.933493e+02, 3.035518e+02, 3.105698e+02, 3.083271e+02, 2.987103e+02, 2.942896e+02 K
EBM (perturbation) finished iteration 24000
2.882894e+02, 2.935439e+02, 3.036881e+02, 3.106732e+02, 3.084462e+02, 2.988803e+02, 2.944895e+02 K
EBM (perturbation) finished iteration 30000
2.883894e+02, 2.936261e+02, 3.037458e+02, 3.107169e+02, 3.084965e+02, 2.989519e+02, 2.945735e+02 K
EBM (perturbation) finished iteration 36000
2.884316e+02, 2.93660

In [33]:
EBM_results_fb_SW_cloud.to_netcdf('nc_from_xarray/EBM_results_noG_fb_sw_cloud.nc')

In [34]:
EBM_results_fb_SW_noncloud = run_EBM_perturb_fb(ds_clim, ds_greg, da_fb_SW_noncloud, models, D=0.96e6)

AM2
Latitudes for test temperature printing:
-71.5
-53.5
-29.5
0.5
30.5
54.5
72.5
EBM (perturbation) finished iteration 0
2.841682e+02, 2.896707e+02, 3.004603e+02, 3.080440e+02, 3.055918e+02, 2.953129e+02, 2.908846e+02 K
EBM (perturbation) finished iteration 6000
2.866128e+02, 2.921982e+02, 3.026326e+02, 3.097466e+02, 3.074509e+02, 2.976489e+02, 2.931195e+02 K
EBM (perturbation) finished iteration 12000
2.885664e+02, 2.938079e+02, 3.036800e+02, 3.104712e+02, 3.083479e+02, 2.990959e+02, 2.949417e+02 K
EBM (perturbation) finished iteration 18000
2.896564e+02, 2.946977e+02, 3.042570e+02, 3.108719e+02, 3.088434e+02, 2.998902e+02, 2.959383e+02 K
EBM (perturbation) finished iteration 24000
2.902526e+02, 2.951864e+02, 3.045766e+02, 3.110950e+02, 3.091174e+02, 3.003238e+02, 2.964781e+02 K
EBM (perturbation) finished iteration 30000
2.905791e+02, 2.954547e+02, 3.047530e+02, 3.112185e+02, 3.092684e+02, 3.005611e+02, 2.967720e+02 K
EBM (perturbation) finished iteration 36000
2.907580e+02, 2.95602

In [35]:
EBM_results_fb_SW_noncloud.to_netcdf('nc_from_xarray/EBM_results_noG_fb_sw_noncloud.nc')

In [36]:
EBM_results_fb_LW_wv = run_EBM_perturb_fb(ds_clim, ds_greg, da_fb_LW_wv, models_noCNRM, D=0.96e6)

AM2
Latitudes for test temperature printing:
-71.5
-53.5
-29.5
0.5
30.5
54.5
72.5
EBM (perturbation) finished iteration 0
2.841682e+02, 2.896707e+02, 3.004603e+02, 3.080440e+02, 3.055918e+02, 2.953129e+02, 2.908846e+02 K
EBM (perturbation) finished iteration 6000
2.858621e+02, 2.913683e+02, 3.018231e+02, 3.091057e+02, 3.067987e+02, 2.968982e+02, 2.923726e+02 K
EBM (perturbation) finished iteration 12000
2.864697e+02, 2.918182e+02, 3.020584e+02, 3.092520e+02, 3.070092e+02, 2.973038e+02, 2.929280e+02 K
EBM (perturbation) finished iteration 18000
2.866120e+02, 2.919219e+02, 3.021115e+02, 3.092846e+02, 3.070564e+02, 2.973959e+02, 2.930548e+02 K
EBM (perturbation) finished iteration 24000
2.866446e+02, 2.919457e+02, 3.021237e+02, 3.092920e+02, 3.070671e+02, 2.974167e+02, 2.930834e+02 K
EBM (perturbation) finished iteration 30000
2.866520e+02, 2.919511e+02, 3.021264e+02, 3.092937e+02, 3.070695e+02, 2.974214e+02, 2.930899e+02 K
EBM (perturbation) finished iteration 36000
2.866537e+02, 2.91952

In [37]:
EBM_results_fb_LW_wv.to_netcdf('nc_from_xarray/EBM_results_noG_fb_lw_wv.nc')

In [38]:
EBM_results_fb_LW_ts = run_EBM_perturb_fb(ds_clim, ds_greg, da_fb_LW_ts, models, D=0.96e6, dt=5.0e-5, nmax=1e6)

AM2
Latitudes for test temperature printing:
-71.5
-53.5
-29.5
0.5
30.5
54.5
72.5
EBM (perturbation) finished iteration 0
2.841681e+02, 2.896705e+02, 3.004600e+02, 3.080436e+02, 3.055916e+02, 2.953127e+02, 2.908845e+02 K
EBM (perturbation) finished iteration 6000
2.856158e+02, 2.913502e+02, 3.020606e+02, 3.093478e+02, 3.069856e+02, 2.968981e+02, 2.921598e+02 K
EBM (perturbation) finished iteration 12000
2.877156e+02, 2.931931e+02, 3.033320e+02, 3.102445e+02, 3.080831e+02, 2.985802e+02, 2.941742e+02 K
EBM (perturbation) finished iteration 18000
2.898164e+02, 2.949258e+02, 3.044601e+02, 3.110298e+02, 3.090708e+02, 3.001585e+02, 2.961356e+02 K
EBM (perturbation) finished iteration 24000
2.917516e+02, 2.965136e+02, 3.054981e+02, 3.117583e+02, 3.099825e+02, 3.015964e+02, 2.979107e+02 K
EBM (perturbation) finished iteration 30000
2.935013e+02, 2.979589e+02, 3.064586e+02, 3.124404e+02, 3.108253e+02, 3.028966e+02, 2.994950e+02 K
EBM (perturbation) finished iteration 36000
2.950765e+02, 2.99270

In [39]:
EBM_results_fb_LW_ts.to_netcdf('nc_from_xarray/EBM_results_noG_fb_lw_ts.nc')

In [40]:
#Make sure I have the sign right. Everything in terms of positive downward, right?
#Yes, defined positive downward in "GregoryTRACMIP". 
#Can always flip sign in plots later, as long as calculations are consistent. 

In [41]:
#For the atmospheric temperature feedback: perturb 5%, 10%, 15%
#
#Naming convention: ##p (for percentage)
#Define different time steps for each model to balance speed vs. convergence

In [42]:
dt_dict_LW_ta_05p = {'AM2':             2.0e-5, 
                     'CAM3':            9.0e-5, 
                     'CAM4':            9.0e-5, 
                     'CNRM-AM6-DIA-v2': 9.0e-5, 
                     'ECHAM-6.1':       9.0e-5, 
                     'ECHAM-6.3':       9.0e-5, 
                     'IPSL-CM5A':       9.0e-5, 
                     'MIROC5':          9.0e-5, 
                     'MPAS':            9.0e-5, 
                     'MetUM-GA6-CTL':   9.0e-5, 
                     'MetUM-GA6-ENT':   9.0e-5, 
                     'NorESM2':         9.0e-5}
EBM_results_fb_LW_ta_05p_AM2_slow = run_EBM_perturb_fb(ds_clim, ds_greg, da_fb_LW_ta, models, 
                                                       D=0.96e6, scale=.05, dt=dt_dict_LW_ta_05p, nmax=2e6)

AM2
Latitudes for test temperature printing:
-71.5
-53.5
-29.5
0.5
30.5
54.5
72.5
EBM (perturbation) finished iteration 0
2.841680e+02, 2.896704e+02, 3.004598e+02, 3.080433e+02, 3.055914e+02, 2.953126e+02, 2.908845e+02 K
EBM (perturbation) finished iteration 6000
2.845894e+02, 2.902570e+02, 3.011525e+02, 3.086685e+02, 3.062042e+02, 2.958949e+02, 2.911753e+02 K
EBM (perturbation) finished iteration 12000
2.851932e+02, 2.909302e+02, 3.017462e+02, 3.091360e+02, 3.067159e+02, 2.965166e+02, 2.917546e+02 K
EBM (perturbation) finished iteration 18000
2.858992e+02, 2.916117e+02, 3.022696e+02, 3.095261e+02, 3.071639e+02, 2.971427e+02, 2.924493e+02 K
EBM (perturbation) finished iteration 24000
2.866412e+02, 2.922742e+02, 3.027411e+02, 3.098686e+02, 3.075705e+02, 2.977523e+02, 2.931740e+02 K
EBM (perturbation) finished iteration 30000
2.873793e+02, 2.929065e+02, 3.031733e+02, 3.101790e+02, 3.079466e+02, 2.983354e+02, 2.938875e+02 K
EBM (perturbation) finished iteration 36000
2.880922e+02, 2.93504

In [43]:
EBM_results_fb_LW_ta_05p_AM2_slow.to_netcdf('nc_from_xarray/EBM_results_noG_fb_lw_ta_05p.nc')

In [56]:
#OK, do the 10% again

#Models that didn't converge originally--try lower dt for them: AM2, ECHAM-6.1, ECHAM-6.3, MetUM-CTL, MetUM-ENT

dt_dict_LW_ta_10p = {'AM2':             2.0e-5, 
                     'CAM3':            9.0e-5, 
                     'CAM4':            9.0e-5, 
                     'CNRM-AM6-DIA-v2': 3.0e-5, 
                     'ECHAM-6.1':       2.0e-5, 
                     'ECHAM-6.3':       2.0e-5, 
                     'IPSL-CM5A':       9.0e-5, 
                     'MIROC5':          9.0e-5, 
                     'MPAS':            9.0e-5, 
                     'MetUM-GA6-CTL':   2.0e-5, 
                     'MetUM-GA6-ENT':   2.0e-5, 
                     'NorESM2':         9.0e-5}
EBM_results_fb_LW_ta_10p = run_EBM_perturb_fb(ds_clim, ds_greg, da_fb_LW_ta, models, 
                                              D=0.96e6, scale=.1, dt=dt_dict_LW_ta_10p, nmax=2e6)


AM2
Latitudes for test temperature printing:
-71.5
-53.5
-29.5
0.5
30.5
54.5
72.5
EBM (perturbation) finished iteration 0
2.841680e+02, 2.896704e+02, 3.004598e+02, 3.080433e+02, 3.055914e+02, 2.953126e+02, 2.908845e+02 K
EBM (perturbation) finished iteration 6000
2.845936e+02, 2.902635e+02, 3.011616e+02, 3.086782e+02, 3.062123e+02, 2.959015e+02, 2.911790e+02 K
EBM (perturbation) finished iteration 12000
2.852160e+02, 2.909600e+02, 3.017800e+02, 3.091677e+02, 3.067454e+02, 2.965456e+02, 2.917775e+02 K
EBM (perturbation) finished iteration 18000
2.859603e+02, 2.916823e+02, 3.023396e+02, 3.095877e+02, 3.072248e+02, 2.972103e+02, 2.925124e+02 K
EBM (perturbation) finished iteration 24000
2.867612e+02, 2.924016e+02, 3.028568e+02, 3.099664e+02, 3.076712e+02, 2.978737e+02, 2.932974e+02 K
EBM (perturbation) finished iteration 30000
2.875771e+02, 2.931047e+02, 3.033428e+02, 3.103185e+02, 3.080945e+02, 2.985238e+02, 2.940892e+02 K
EBM (perturbation) finished iteration 36000
2.883841e+02, 2.93785

In [57]:
EBM_results_fb_LW_ta_10p.to_netcdf('nc_from_xarray/EBM_results_noG_fb_lw_ta_10p.nc')

In [46]:
#Try 15% now

dt_dict_LW_ta_15p = {'AM2':             2.3e-5, 
                     'CAM3':            9.0e-5, 
                     'CAM4':            9.0e-5, 
                     'CNRM-AM6-DIA-v2': 2.3e-5, 
                     'ECHAM-6.1':       2.3e-5, 
                     'ECHAM-6.3':       2.3e-5, 
                     'IPSL-CM5A':       2.3e-5, 
                     'MIROC5':          9.0e-5, 
                     'MPAS':            9.0e-5, 
                     'MetUM-GA6-CTL':   2.0e-5, 
                     'MetUM-GA6-ENT':   2.0e-5, 
                     'NorESM2':         9.0e-5}
EBM_results_fb_LW_ta_15p = run_EBM_perturb_fb(ds_clim, ds_greg, da_fb_LW_ta, models, 
                                              D=0.96e6, scale=.15, dt=dt_dict_LW_ta_15p, nmax=3e6)


AM2
Latitudes for test temperature printing:
-71.5
-53.5
-29.5
0.5
30.5
54.5
72.5
EBM (perturbation) finished iteration 0
2.841680e+02, 2.896704e+02, 3.004598e+02, 3.080434e+02, 3.055915e+02, 2.953126e+02, 2.908845e+02 K
EBM (perturbation) finished iteration 6000
2.846820e+02, 2.903735e+02, 3.012711e+02, 3.087704e+02, 3.063079e+02, 2.960046e+02, 2.912602e+02 K
EBM (perturbation) finished iteration 12000
2.854632e+02, 2.912176e+02, 3.019986e+02, 3.093412e+02, 3.069333e+02, 2.967845e+02, 2.920228e+02 K
EBM (perturbation) finished iteration 18000
2.864066e+02, 2.921072e+02, 3.026720e+02, 3.098445e+02, 3.075118e+02, 2.976053e+02, 2.929557e+02 K
EBM (perturbation) finished iteration 24000
2.874278e+02, 2.930073e+02, 3.033109e+02, 3.103131e+02, 3.080665e+02, 2.984387e+02, 2.939551e+02 K
EBM (perturbation) finished iteration 30000
2.884765e+02, 2.939025e+02, 3.039281e+02, 3.107633e+02, 3.086075e+02, 2.992695e+02, 2.949707e+02 K
EBM (perturbation) finished iteration 36000
2.895254e+02, 2.94785

In [47]:
EBM_results_fb_LW_ta_15p.to_netcdf('nc_from_xarray/EBM_results_noG_fb_lw_ta_15p.nc')
#Wait--had originally used scale = .1 again. 
#Will 15% actually work?

#ECHAM6.3 had an overflow--unrealistic anyway, reached like 350 K (gradually, not abruptly): seems like it's blowing up. OK to return NaNs

#IPSL blew up (10^6-ish)
#Maybe have another exit condition for very high temperature? But can't tell if it's numerical problem or runaway feedback

#Check what results look like without that one, anyway, while redoing the IPSL one?

In [48]:

#####   LW CLOUD ADJUSTMENT EXPERIMENTS   #####


In [49]:
#Loading of Gregory results slotted in with the rest of the terms above

In [50]:
EBM_results_adj_LW_cloud = run_EBM_perturb_adj(ds_clim, ds_greg, da_adj_LW_cloud, models_10, D=0.96e6)

AM2
Latitudes for test temperature printing:
-71.5
-53.5
-29.5
0.5
30.5
54.5
72.5
EBM (perturbation) finished iteration 0
2.841681e+02, 2.896706e+02, 3.004603e+02, 3.080436e+02, 3.055918e+02, 2.953128e+02, 2.908844e+02 K
EBM (perturbation) finished iteration 6000
2.863928e+02, 2.921698e+02, 3.027754e+02, 3.098417e+02, 3.075399e+02, 2.975805e+02, 2.928336e+02 K
EBM (perturbation) finished iteration 12000
2.888002e+02, 2.941587e+02, 3.040794e+02, 3.107642e+02, 3.086731e+02, 2.993838e+02, 2.950875e+02 K
EBM (perturbation) finished iteration 18000
2.904358e+02, 2.954960e+02, 3.049552e+02, 3.113895e+02, 3.094437e+02, 3.006043e+02, 2.966099e+02 K
EBM (perturbation) finished iteration 24000
2.915222e+02, 2.963903e+02, 3.055496e+02, 3.118182e+02, 3.099677e+02, 3.014199e+02, 2.976160e+02 K
EBM (perturbation) finished iteration 30000
2.922460e+02, 2.969891e+02, 3.059516e+02, 3.121100e+02, 3.103221e+02, 3.019648e+02, 2.982829e+02 K
EBM (perturbation) finished iteration 36000
2.927294e+02, 2.97390

In [51]:
EBM_results_fb_LW_cloud = run_EBM_perturb_fb(ds_clim, ds_greg, da_fb_LW_cloud, models_10, D = 0.96e6)

AM2
Latitudes for test temperature printing:
-71.5
-53.5
-29.5
0.5
30.5
54.5
72.5
EBM (perturbation) finished iteration 0
2.841682e+02, 2.896707e+02, 3.004603e+02, 3.080440e+02, 3.055918e+02, 2.953129e+02, 2.908846e+02 K
EBM (perturbation) finished iteration 6000
2.866379e+02, 2.922313e+02, 3.026035e+02, 3.097386e+02, 3.074886e+02, 2.977936e+02, 2.932932e+02 K
EBM (perturbation) finished iteration 12000
2.886365e+02, 2.938873e+02, 3.036619e+02, 3.104884e+02, 3.084719e+02, 2.994643e+02, 2.954207e+02 K
EBM (perturbation) finished iteration 18000
2.897877e+02, 2.948364e+02, 3.042733e+02, 3.109290e+02, 3.090538e+02, 3.004454e+02, 2.966639e+02 K
EBM (perturbation) finished iteration 24000
2.904469e+02, 2.953831e+02, 3.046302e+02, 3.111889e+02, 3.093950e+02, 3.010120e+02, 2.973750e+02 K
EBM (perturbation) finished iteration 30000
2.908269e+02, 2.956991e+02, 3.048381e+02, 3.113409e+02, 3.095935e+02, 3.013382e+02, 2.977819e+02 K
EBM (perturbation) finished iteration 36000
2.910465e+02, 2.95882

In [52]:
#Converged for every model in both cases. Now save to NetCDF files

In [53]:
EBM_results_adj_LW_cloud.to_netcdf('nc_from_xarray/EBM_results_noG_adj_lw_cloud.nc')

In [54]:
EBM_results_fb_LW_cloud.to_netcdf('nc_from_xarray/EBM_results_noG_fb_lw_cloud.nc')